In [ ]:
# import sys
# prefix='/pi/ws/sagas-ai/bots/agent_dispatcher'
# sys.path.append(prefix)

* 通过语法解析来进行意图推断, 并提取相应的参数

In [1]:
from sagas.kit.rulesets_kit import RulesetsKit
from sagas.nlu.inspector_registry import ci
from pprint import pprint
sents='you took fifty damage'
lang='en'
prefix='.'
rs = RulesetsKit().execute(f"{prefix}/assets/rs_common_{lang}.yml",
                           test_intent=None,
                           test_sents=sents,
                           show_graph=False)
rs

cust_ents inspectors.cust_entity_inspector.CustEntityInspector
✁ ------------------------- r1
(en) you took fifty damage
verb_domains 2 took take ['rel', 'lemma', 'word', 'index', 'stems', 'lang', 'sents']
[('nsubj', '1', 'you', 'you', ['you'], ['c_pron', 'x_prp']),
 ('obj', '4', 'damage', 'damage', ['fifty', 'damage'], ['c_noun', 'x_nn']),
 ('punct', '5', '.', '.', ['.'], ['c_punct', 'x_.'])]
✖ (food) root with obl is ins_cust_ents(cuisine): False
¤food¤ result: False
✁ ------------------------- r1
(en) you took fifty damage
verb_domains 2 took take ['rel', 'lemma', 'word', 'index', 'stems', 'lang', 'sents']
[('nsubj', '1', 'you', 'you', ['you'], ['c_pron', 'x_prp']),
 ('obj', '4', 'damage', 'damage', ['fifty', 'damage'], ['c_noun', 'x_nn']),
 ('punct', '5', '.', '.', ['.'], ['c_punct', 'x_.'])]
.. damage/damage is damage: True
✔ (injured) verb with pos is behave_of(take,v): True, pos is ins_path(any obj is damage@n): True, obj is Inspector('ins_date'): True
.. results 2
{'behave_of/d

{'behave_of/default/...': {'category': 'take', 'pos': 'v'},
 'ins_date/duckling/...': ''}

¤injured¤ result: True
✁ ------------------------- r1
(en) you took fifty damage
verb_domains 2 took take ['rel', 'lemma', 'word', 'index', 'stems', 'lang', 'sents']
[('nsubj', '1', 'you', 'you', ['you'], ['c_pron', 'x_prp']),
 ('obj', '4', 'damage', 'damage', ['fifty', 'damage'], ['c_noun', 'x_nn']),
 ('punct', '5', '.', '.', ['.'], ['c_punct', 'x_.'])]
✖ (dead) cop with pos is behave_of(dead,a): False, nsubj is ['c_pron', 'c_noun', 'c_propn']: True, cop is c_aux: False
¤dead¤ result: False


[{'intent': 'food', 'result': False, 'data': []},
 {'intent': 'injured',
  'result': True,
  'data': [{'inspector': 'behave_of',
    'provider': 'default',
    'part': 'predicate',
    'value': {'category': 'take', 'pos': 'v'},
    'delivery': 'sentence'},
   {'inspector': 'ins_date',
    'provider': 'duckling',
    'part': 'obj',
    'value': [{'body': 'fifty',
      'start': 0,
      'value': {'value': 50, 'type': 'value'},
      'end': 5,
      'dim': 'number',
      'latent': False}],
    'delivery': 'slot'}]},
 {'intent': 'dead', 'result': False, 'data': []}]

* 过滤掉不匹配的意图

In [2]:
intents=[intent for intent in rs if intent['result']]
intents

[{'intent': 'injured',
  'result': True,
  'data': [{'inspector': 'behave_of',
    'provider': 'default',
    'part': 'predicate',
    'value': {'category': 'take', 'pos': 'v'},
    'delivery': 'sentence'},
   {'inspector': 'ins_date',
    'provider': 'duckling',
    'part': 'obj',
    'value': [{'body': 'fifty',
      'start': 0,
      'value': {'value': 50, 'type': 'value'},
      'end': 5,
      'dim': 'number',
      'latent': False}],
    'delivery': 'slot'}]}]

* 在意图匹配测试时, inspectors同时进行值测试和参数提取, 提取结果的数据结构较复杂
* 所以需要根据需要进行过滤/修剪; 使用函数映射表来做这个工作, 默认只提取意图名

In [3]:
def digest_injured(intent):
    from jsonpath_ng import jsonpath, parse    
    jsonpath_expr = parse('data[*].value[*].value.value')
    val=next(match.value for match in jsonpath_expr.find(intent))
    return {'value':val}

# intent -> filter/extractor
intent_procs={'injured':digest_injured}

resp_msgs=[]
for intent in intents:
    intent_name=intent['intent']
    resp_msg={'intent':intent_name}  # default extractor
    
    if intent_name in intent_procs:
        resp_msg.update(intent_procs[intent_name](intent))
    resp_msgs.append(resp_msg)
resp_msgs

[{'intent': 'injured', 'value': 50}]

* 关于jsonpath_ng, 参考:
⊕ [jsonpath-ng · PyPI](https://pypi.org/project/jsonpath-ng/)

In [2]:
from sagas.kit.rulesets_kit import RulesetsKit
from sagas.nlu.inspector_registry import ci
from pprint import pprint
sents='you took fifty damage'
lang='en'
prefix='.'
kit=RulesetsKit('rasa')
rs = kit.execute(f"{prefix}/assets/rs_common_{lang}.yml",
                           test_intent=None,
                           test_sents=sents,
                           show_graph=False)
# rs
ok_list=[intent for intent in rs if intent['result']]
ok_list

.. with extractor rasa
✁ ------------------------- r4
(en) you took fifty damage
verb_domains 2 took take ['rel', 'lemma', 'word', 'index', 'stems', 'lang', 'sents']
[('nsubj', '1', 'you', 'you', ['you'], ['c_pron', 'x_prp']),
 ('obj', '4', 'damage', 'damage', ['fifty', 'damage'], ['c_noun', 'x_nn']),
 ('punct', '5', '.', '.', ['.'], ['c_punct', 'x_.'])]
✖ (food) root with obl is ins_cust_ents(cuisine): False
¤food¤ result: False
✁ ------------------------- r5
(en) you took fifty damage
verb_domains 2 took take ['rel', 'lemma', 'word', 'index', 'stems', 'lang', 'sents']
[('nsubj', '1', 'you', 'you', ['you'], ['c_pron', 'x_prp']),
 ('obj', '4', 'damage', 'damage', ['fifty', 'damage'], ['c_noun', 'x_nn']),
 ('punct', '5', '.', '.', ['.'], ['c_punct', 'x_.'])]
.. damage/damage is damage: True
✔ (injured) verb with pos is behave_of(take,v): True, pos is ins_path(any obj is damage@n): True, obj is Inspector('ins_date'): True
.. results 2
{'behave_of/default/predicate', 'ins_date/duckling/ob

{'behave_of/default/...': {'category': 'take', 'pos': 'v'},
 'ins_date/duckling/...': ''}

¤injured¤ result: True
✁ ------------------------- r5
(en) you took fifty damage
verb_domains 2 took take ['rel', 'lemma', 'word', 'index', 'stems', 'lang', 'sents']
[('nsubj', '1', 'you', 'you', ['you'], ['c_pron', 'x_prp']),
 ('obj', '4', 'damage', 'damage', ['fifty', 'damage'], ['c_noun', 'x_nn']),
 ('punct', '5', '.', '.', ['.'], ['c_punct', 'x_.'])]
✖ (dead) cop with pos is behave_of(dead,a): False, nsubj is ['c_pron', 'c_noun', 'c_propn']: True, cop is c_aux: False
¤dead¤ result: False


[{'intent': 'injured',
  'result': True,
  'priority': 5,
  'confidence': 1.0,
  'text': 'you took fifty damage',
  'data': [{'confidence': None,
    'start': 4,
    'end': 8,
    'entity': 'verb_domains',
    'extractor': 'ruleset',
    'value': 'took/take',
    'additional_info': [{'inspector': 'behave_of',
      'provider': 'default',
      'part': 'predicate',
      'value': {'category': 'take', 'pos': 'v'},
      'delivery': 'sentence'},
     {'inspector': 'ins_date',
      'provider': 'duckling',
      'part': 'obj',
      'value': [{'body': 'fifty',
        'start': 0,
        'value': {'value': 50, 'type': 'value'},
        'end': 5,
        'dim': 'number',
        'latent': False}],
      'delivery': 'slot'}]}]}]

In [6]:
rankings=sorted(ok_list, key=lambda e: e['confidence'], reverse=True)
rankings

[{'intent': 'injured',
  'result': True,
  'priority': 5,
  'confidence': 1.0,
  'text': 'you took fifty damage',
  'data': [{'confidence': None,
    'start': 4,
    'end': 8,
    'entity': 'verb_domains',
    'extractor': 'ruleset',
    'value': 'took/take',
    'additional_info': [{'inspector': 'behave_of',
      'provider': 'default',
      'part': 'predicate',
      'value': {'category': 'take', 'pos': 'v'},
      'delivery': 'sentence'},
     {'inspector': 'ins_date',
      'provider': 'duckling',
      'part': 'obj',
      'value': [{'body': 'fifty',
        'start': 0,
        'value': {'value': 50, 'type': 'value'},
        'end': 5,
        'dim': 'number',
        'latent': False}],
      'delivery': 'slot'}]}]}]

In [12]:
rankings[1:]

[]

In [14]:
rank=rankings[0]
{
    'intent': {'confidence': rank['confidence'], 'name': rank['intent']},
    'text': rank['text'],
    'entities': rank['data'],
    'intent_ranking': [{'confidence': e['confidence'], 'name': e['intent']} for e in rankings[1:]],
}

{'intent': {'confidence': 1.0, 'name': 'injured'},
 'text': 'you took fifty damage',
 'entities': [{'confidence': None,
   'start': 4,
   'end': 8,
   'entity': 'verb_domains',
   'extractor': 'ruleset',
   'value': 'took/take',
   'additional_info': [{'inspector': 'behave_of',
     'provider': 'default',
     'part': 'predicate',
     'value': {'category': 'take', 'pos': 'v'},
     'delivery': 'sentence'},
    {'inspector': 'ins_date',
     'provider': 'duckling',
     'part': 'obj',
     'value': [{'body': 'fifty',
       'start': 0,
       'value': {'value': 50, 'type': 'value'},
       'end': 5,
       'dim': 'number',
       'latent': False}],
     'delivery': 'slot'}]}],
 'intent_ranking': []}

In [3]:
kit.intent_matched

['injured']

In [16]:
import io_utils
io_utils.list_with_suffix('.', 'py')

['./__init__.py',
 './servants/avatar_blueprints.py',
 './servants/__init__.py',
 './servants/avatar_servant.py',
 './actions/actions.py',
 './actions/__init__.py',
 './inspectors/cust_entity_inspector.py',
 './inspectors/__init__.py',
 './rulesets/rules_es.py',
 './rulesets/__init__.py']

In [1]:
import io_utils
io_utils.list_match('.', '*.py')

['./__init__.py',
 './servants/avatar_blueprints.py',
 './servants/__init__.py',
 './servants/avatar_servant.py',
 './actions/actions.py',
 './actions/__init__.py',
 './inspectors/cust_entity_inspector.py',
 './inspectors/__init__.py',
 './rulesets/rules_es.py',
 './rulesets/__init__.py']